In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
zip_path = '/content/gdrive/My Drive/Data.zip'
!unzip '$zip_path' 
!unzip -q '/content/SIMS dataset/images.zip' 
!unzip -q '/content/SIMS dataset/Annotations_in_3_formats.zip' 

Archive:  /content/gdrive/My Drive/Data.zip
  inflating: SIMS dataset/test.txt   
  inflating: SIMS dataset/validation.txt  
  inflating: SIMS dataset/training.txt  
  inflating: SIMS dataset/Assignment-2_modified scope.pdf  
  inflating: SIMS dataset/Annotations_in_3_formats.zip  
  inflating: SIMS dataset/images.zip  


In [0]:
%cp -R "/content/annotations/pascal_voc/Annotations/."  "/content/images"


In [0]:
!mkdir Dataset
!mkdir Dataset/train
!mkdir Dataset/train/images
!mkdir Dataset/train/annotations
!mkdir Dataset/valid
!mkdir Dataset/valid/images
!mkdir Dataset/valid/annotations
!mkdir Dataset/test
!mkdir Dataset/test/images
!mkdir Dataset/test/annotations

In [0]:
import shutil
import os
#copy files based on {datasets in txt}

def copy(filepath,dest_dir):
    with open(filepath) as fp:
        for line in fp:
            os.makedirs(dest_dir, exist_ok=True)  # succeeds even if directory exists.
            shutil.copy(line.replace('\n', ''), dest_dir+"/images") #image
            shutil.copy(line.replace('.jpg\n', '.xml'), dest_dir+"/annotations") ##label

#copy files based on {datasets in txt}
copy('/content/SIMS dataset/training.txt','Dataset/train/')
copy('/content/SIMS dataset/validation.txt','Dataset/valid/')
copy('/content/SIMS dataset/test.txt','Dataset/test/')

#from preprocessing.pascal_voc_to_csv import xml_to_csv

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path,image_path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        #print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text
            
            size = root.find('size') #contains height and width of image
            w = int(size.find('width').text)
            h = int(size.find('height').text)
            
            # truncate any bb coordinates that fall outside of the image
            xmin = max(0, xmin)
            ymin = max(0, ymin)
            xmax = min(w, xmax)
            ymax = min(h, ymax)
            
            # # ignore the bounding boxes where the minimum values are larger
            # # than the maximum values and vice-versa due to annotation errors
            if xmin >= xmax or ymin >= ymax:
                continue
            elif xmax <= xmin or ymax <= ymin:
                continue
            
            file_path = image_path+root.find('filename').text+".jpg"
            value = (file_path,
                     xmin,
                     ymin,
                     xmax,
                     ymax,
                     label,
                     )
            xml_list.append(value)
    column_name = ['filename','xmin', 'ymin', 'xmax', 'ymax','class']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    root_path = "Dataset/"
    datasets = ['train','test','valid']
    for ds in datasets:
        #image_path = os.path.join(os.getcwd(), root_path+ds, 'annotations')
        image_path = os.path.join(root_path+ds, 'annotations/')
        print(image_path)
        update_path = os.path.join(root_path+ds, 'images/')
        xml_df = xml_to_csv(image_path,update_path)
        xml_df.to_csv('labels_{}.csv'.format(ds), index=None, header=False)
        print('Successfully converted xml to csv.')

In [6]:
!git clone https://github.com/arslanamin14/keras-yolo3.git

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [7]:
%cd /content/keras-yolo3/
!ls

/content/keras-yolo3
callbacks.py		  README.md
config.json		  requirements.txt
create_validation_set.sh  train.py
evaluate.py		  utils
gen_anchors.py		  voc.py
generator.py		  yolo3_one_file_to_detect_them_all.py
LICENSE			  yolo.py
predict.py		  zoo


In [8]:
!pip install -r requirements.txt

     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 2.4MB 14.9MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 1.3MB 52.4MB/s 
     |████████████████████████████████| 276kB 53.6MB/s 
     |████████████████████████████████| 3.8MB 54.1MB/s 
     |████████████████████████████████| 412.3MB 44kB/s 
     |████████████████████████████████| 512kB 45.4MB/s 
     |████████████████████████████████| 327kB 60.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b76d3429edc2c29c997e996f02f4d7c02b03bb08f6bc8d06c310ee1ae2ff7f80
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp36-none-any.whl size=61682 sha256=0f7decf3b88869586298dc3e1e49a8d003000abbdf

In [9]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 37kB/s 


In [0]:
%cp -R "/content/gdrive/My Drive/folder/backend.h5"  "/content/keras-yolo3" 

In [0]:
%cp -R "/content/gdrive/My Drive/config.json"  "/content/keras-yolo3" 

In [13]:
!cd "/content/keras-yolo3" && python gen_anchors.py -c config.json

/content/Dataset/valid/images/0011
/content/Dataset/valid/images/0013
/content/Dataset/valid/images/0014
/content/Dataset/valid/images/0015
/content/Dataset/valid/images/0020
/content/Dataset/valid/images/0023
/content/Dataset/valid/images/0025
/content/Dataset/valid/images/0033
/content/Dataset/valid/images/0035
/content/Dataset/valid/images/0036
/content/Dataset/valid/images/0042
/content/Dataset/valid/images/0045
/content/Dataset/valid/images/0049
/content/Dataset/valid/images/0054
/content/Dataset/valid/images/0055
/content/Dataset/valid/images/0062
/content/Dataset/valid/images/0067
/content/Dataset/valid/images/0070
/content/Dataset/valid/images/0082
/content/Dataset/valid/images/0087
/content/Dataset/valid/images/0088
/content/Dataset/valid/images/0119
/content/Dataset/valid/images/0122
/content/Dataset/valid/images/0125
/content/Dataset/valid/images/0126
/content/Dataset/valid/images/0130
/content/Dataset/valid/images/0134
/content/Dataset/valid/images/0136
/content/Dataset/val

In [13]:
!cd "/content/keras-yolo3" && python train.py -c config.json

Streaming output truncated to the last 5000 lines.
resizing:  128 128
 - 579s - loss: 15.1054 - yolo_layer_1_loss: 0.4948 - yolo_layer_2_loss: 2.2978 - yolo_layer_3_loss: 12.3128

Epoch 00013: loss improved from 15.36180 to 15.10545, saving model to /content/drive/My Drive/yolo/yolo3.h5
resizing:  128 128
resizing:  128 128
Epoch 14/28
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 1

In [15]:
!cd "/content/keras-yolo3" && python evaluate.py -c config.json

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-05-15 21:33:25.200864: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-15 21:33:25.221592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-15 21:33:25.222139: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-05-15 21:33:25.222400: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-05-15 21:33:25.223661: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-05-15 21:33:25.22477

In [16]:
!cd "/content/keras-yolo3" && python predict.py -c config.json -i "/content/Dataset/test/images/0045.jpg"

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-05-15 21:37:52.354540: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-15 21:37:52.371595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-15 21:37:52.372150: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-05-15 21:37:52.372470: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-05-15 21:37:52.373715: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-05-15 21:37:52.38324